In [1]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.


# Explanation of the Solution:

## Data Loading:

We use torchvision.datasets.MNIST to load the dataset and apply transformations (normalization).
The DataLoader creates batches of 64 images, which is efficient for training.

## Neural Network Architecture:

We define a simple feedforward neural network (SimpleNN) with one hidden layer (256 neurons) and ReLU activation.
The input size is 28x28=784 (flattened), and the output size is 10 (for 10 classes, digits 0–9).


## Training Loop:

The loss function is cross-entropy loss, which is appropriate for multi-class classification.
The optimizer is Adam, which dynamically adjusts the learning rate and typically works well for this task.
We run the training loop for 5 epochs, printing the average loss at each epoch.


## Evaluation:

After training, we evaluate the model on the test set, calculating accuracy based on the number of correct predictions.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Step 1: Data Loading
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Step 2: Define the Neural Network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)  # Input layer (784 nodes) to hidden layer (256 nodes)
        self.fc2 = nn.Linear(256, 10)  # Hidden layer (256 nodes) to output layer (10 nodes)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the 28x28 image to a 784-dim vector
        x = torch.relu(self.fc1(x))  # Apply ReLU activation function to the hidden layer
        x = self.fc2(x)  # Output layer (no activation function needed for cross-entropy)
        return x

# Instantiate the model
model = SimpleNN()

# Step 3: Define the Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Step 4: Training Loop
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate loss for monitoring
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# Step 5: Evaluate the Model
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test dataset: {accuracy:.2f}%')


Epoch 1/5, Loss: 0.3381021556092986
Epoch 2/5, Loss: 0.16017595110441257
Epoch 3/5, Loss: 0.11426883577796093
Epoch 4/5, Loss: 0.09084531569084935
Epoch 5/5, Loss: 0.07809582434997742
Accuracy of the model on the test dataset: 97.37%
